In [ ]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate

In [ ]:
from google.colab import userdata
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, TextStreamer, BitsAndBytesConfig
import torch


In [ ]:
hf_token=userdata.get("project")

In [ ]:
login(hf_token,add_to_git_credential=True)

In [ ]:
LLAMA="meta-llama/Llama-3.1-8B-Instruct"
PHI3="microsoft/Phi-3-mini-4k-instruct"
GEMMA2="google/gemma-2-2b-it"
QUEN2="Qwen/Qwen2.5-7B-Instruct"
MIXTRAL="mistralai/Mixtral-8x7B-Instruct-v0.1"

In [ ]:
messages=[
    {"role":"system","content":"You ar  a helpful assistant"},
    {"role":"user","content":"What is the meaning of life?"}
]

In [ ]:
# qunatization configuration-this allows us to load the model into memory and use less memory
#tHIS THING IS REALLY USEFUL AT MANY PLACES
quant_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16)

In [ ]:
#Tokenizer
tokenizer=AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token=tokenizer.eos_token#Okten used to fill up the prompt of ours
inputs=tokenizer.apply_chat_template(messages,return_tensors="pt")


In [ ]:
pip install -U bitsandbytes

In [ ]:
#the model
model=AutoModelForCausalLM.from_pretrained(LLAMA,device_map="auto",)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
memory=model.get_memory_footprint()/1e6

In [ ]:
print(f"Memory footprint fo the model is {memory} MB")

Memory footprint fo the model is 32121.045248 MB


In [ ]:
model #Actual deep neural netrok fo  our model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "CPU")
model.to(device)

inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)

outputs = model.generate(inputs, max_new_tokens=80)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
messages1=[
    {"role":"system","content":"Hi assisant"},
    {"role":"user","content":"tell me a joke?"}
]

In [ ]:


inputs = tokenizer.apply_chat_template(messages1, return_tensors="pt").to(device)

outputs = model.generate(inputs, max_new_tokens=80)
print(tokenizer.decode(outputs[0]))

In [ ]:
for var in ['inputs', 'outputs', 'model']:
    if var in locals():
        del globals()[var]

torch.cuda.empty_cache()

In [ ]:
#Wrapping everything  in a function and adding Streamling
def generate(model,messages):
    tokenizer=AutoTokenizer.from_pretrained(LLAMA)
    tokenizer.pad_token=tokenizer.eos_token
    model=AutoModelForCausalLM.from_pretrained(LLAMA,quantization_config=quant_config,device_map="auto")
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    outputs=model.generate(input ,max_new_tokens=80,streamer=streamer)
    del tokenizer,streamer,model,inputs,outputs
    torch.cuda.empty_cache()

In [ ]:
generate(PHI3,messages)